# 🥣 Lesson 1: Mangkuk ke Bukan?

**Masa:** 60 minit

**Goal:** Buat AI yang boleh kenal perbezaan gambar - collect data sendiri dari internet!

---

## 🤯 The "Impossible" Became Easy!

### xkcd Comic (2015)

Pada tahun **2015**, ada satu joke dalam xkcd comic:

> *"Dalam 5 tahun lalu, kalau boss suruh kau check sama ada gambar tu bird atau bukan... itu memang IMPOSSIBLE!"*

Recognize bird dari gambar dianggap **so impossible** sampai jadi bahan lawak! 🤣

### Fast Forward ke Hari Ni...

**Jeremy Howard** (pencipta fastai) buat **bird classifier dalam 2 minit** guna laptop biasa. Bukan supercomputer. Bukan jutaan ringgit. Just laptop presentation dia!

```
2015: "Recognize bird? IMPOSSIBLE! 😱"
2024: "Recognize bird? 2 minit je! 😎"
```

**Apa yang dah berubah?**
- ✅ Deep Learning algorithms
- ✅ Pre-trained models (transfer learning)
- ✅ Libraries macam fastai & PyTorch
- ✅ GPU yang affordable

> *"Something pretty extraordinary has happened since 2015... literally something that went from so impossible it's a joke, to so easy that I can run it on my laptop in about two minutes."* - Jeremy Howard

---

## 🤔 Kenapa Belajar Ni?

Korang tau tak:
- **Instagram/TikTok filters** - detect muka korang guna AI
- **Grab Food** - scan gambar makanan untuk categorize
- **CCTV** - detect orang/kereta

Semua ni guna **Image Classification**! Jom buat sendiri! 🔥

### 🎯 Hari Ni Kita Belajar:
1. **Search** gambar dari internet (guna DuckDuckGo)
2. **Download** gambar tu
3. **Clean** data (buang gambar rosak)
4. **Train** AI model!

Sama macam **Jeremy Howard** dalam course.fast.ai! 🚀

> **Note:** DuckDuckGo sometimes rate limits. If search fails, try: Runtime > Restart runtime, then run again.

---

## 🖥️ Sejarah Ringkas: Dari Asap ke AI

### 🔥 Zaman Purba: Komunikasi Asap
Dulu, manusia gua berkomunikasi guna **asap api**:
- Asap naik = "Ada bahaya!"
- Tak ada asap = "Selamat"

Ni sebenarnya **binary** pertama dunia - 2 keadaan je: **ADA** atau **TAKDA**!

### 💡 Zaman Komputer: Binary (1 dan 0)
Komputer moden guna konsep sama:
- `1` = ada elektrik (ON)
- `0` = takda elektrik (OFF)

Semua yang korang nampak di screen ni - gambar, video, TikTok - semuanya cuma **1 dan 0**!

```
Contoh huruf "A" dalam binary:
01000001
```

### 🖼️ Gambar = Nombor! (Images are Made of Numbers)

Komputer tak nampak gambar macam kita. Komputer nampak **NOMBOR** je!

Setiap pixel dalam gambar ada **3 nilai** (RGB):
- **R** = Red (merah) → 0 sampai 255
- **G** = Green (hijau) → 0 sampai 255
- **B** = Blue (biru) → 0 sampai 255

**Contoh:** Paruh burung warna oren mungkin:
```
R = 251 (banyak merah)
G = 48  (sikit hijau)
B = 21  (sikit biru)
```

**Satu gambar 256×171 pixels sebenarnya:**
```
256 × 171 × 3 = 131,328 nombor!
```

> *"So this picture, the thing that we recognize as a picture, is actually 256 × 171 × 3 numbers, between 0 and 255, representing the amount of red, green and blue on each pixel."* - Jeremy Howard

**Ini yang AI "nampak":**
```
┌─────────────────────────────┐
│ [251, 48, 21] [250, 47, 20] │  ← Paruh burung (oren)
│ [120, 180, 80] [118, 182, 82]│  ← Daun (hijau)
│ [50, 150, 220] [52, 148, 218]│  ← Langit (biru)
│ ...                         │
└─────────────────────────────┘
```

Jadi bila kita train AI untuk kenal gambar, sebenarnya kita train AI untuk faham **patterns dalam nombor-nombor ni**! 🔢

### 🗣️ Evolusi Programming Languages

| Level | Language | Contoh | Analogi |
|-------|----------|--------|---------|
| **Paling Low** | Machine Code | `01001000 01101001` | Cakap terus dengan otak komputer |
| **Low Level** | Assembly | `MOV AX, 5` | Cakap bahasa robot |
| **Mid Level** | C, C++ | `printf("Hello")` | Cakap bahasa engineer |
| **High Level** | Python, JavaScript | `print("Hello")` | Cakap macam manusia! |
| **Very High** | fastai | `learn.fine_tune(3)` | Cakap macam "tolong train AI ni" |

### 🐍 Kenapa Python?

**Python** (dicipta 1991) popular sebab:
- ✅ Senang dibaca (macam English!)
- ✅ Banyak library siap pakai
- ✅ Best untuk AI/Machine Learning
- ✅ Digunakan oleh Google, Netflix, Instagram

```python
# Python senang dibaca:
if umur >= 18:
    print("Boleh mengundi!")
```

### 🧠 Kenapa Neural Networks Hebat? (Automatic Feature Learning)

Dulu (sebelum deep learning), untuk buat AI kenal gambar, team saintis kena **manually code** setiap feature:
- "Kalau ada garisan zig-zag begini = bird feather"
- "Kalau ada bulatan merah = apple"
- ... (thousands of rules!)

**Ini memakan TAHUN dan ramai pakar!**

### ✨ Neural Networks Belajar SENDIRI!

Pada 2015, **Zeiler & Fergus** tengok "dalam" neural network dan jumpa sesuatu yang amazing:

```
LAYER 1: Belajar BASIC features
┌─────────────────────────────────────────┐
│ ╱  ╲  │  ─   🟡🔵🟢  (edges, gradients) │
└─────────────────────────────────────────┘
          ↓ combine...

LAYER 2: Belajar MEDIUM features  
┌─────────────────────────────────────────┐
│ ∟  ⌒  ○  ◐   (corners, curves, circles)│
└─────────────────────────────────────────┘
          ↓ combine...

LAYER 3: Belajar COMPLEX features
┌─────────────────────────────────────────┐
│ 🌸 👁️ 🔤  (flower petals, eyes, text)   │
└─────────────────────────────────────────┘
          ↓ combine...

FINAL: Recognize whole objects!
┌─────────────────────────────────────────┐
│ 🐦 = Bird!  🐱 = Cat!  🚗 = Car!         │
└─────────────────────────────────────────┘
```

**Mind-blowing part:**
- Neural network discover these features **SENDIRI**!
- Kita tak perlu code "cari edge" atau "cari circle"
- Neural network automatically learns dari data! 🤯

> *"We don't have to hand code any of these... You just start with a random neural network and feed it examples, and it turns out these are the things that it creates for itself."* - Jeremy Howard

**Ini sebab deep learning boleh buat benda yang sebelum ni IMPOSSIBLE!**

### 🎯 Pyramid of AI Languages

```
        ┌─────────────┐
        │   fastai    │  ← Kita guna ni! (Paling senang)
        │ "Train AI"  │
        ├─────────────┤
        │   PyTorch   │  ← Library AI (Google/Meta guna)
        │  Tensors    │
        ├─────────────┤
        │   Python    │  ← Programming language
        │             │
        ├─────────────┤
        │   C / C++   │  ← Low level, laju
        │             │
        ├─────────────┤
        │  Assembly   │  ← Susah gila!
        │             │
        └─────────────┘
        │  Binary     │  ← 1 dan 0
        │ 01010101... │
```

**Analogi:**
- **Binary** = Batu-bata
- **C/C++** = Simen & kayu
- **Python** = Rumah siap separuh
- **PyTorch** = Rumah dengan wiring elektrik
- **fastai** = Rumah fully furnished! Masuk je duduk! 🏠

> **Jeremy Howard** (pencipta fastai) kata: *"Kita nak orang focus pada SOLVE PROBLEM, bukan tulis code yang susah-susah."*

---

### 📓 Apa Itu Jupyter Notebook?

**Jupyter Notebook** = "Buku nota interaktif" untuk coding!

Bayangkan macam **buku latihan sekolah**:
- Ada bahagian **tulis nota** (Markdown cells) ← Macam ni!
- Ada bahagian **buat kerja** (Code cells) ← Boleh run code!
- Semua dalam **satu file** `.ipynb`

#### 🤔 Kenapa Guna Notebook?

| Traditional `.py` file | Jupyter Notebook `.ipynb` |
|------------------------|---------------------------|
| Code je, takda explanation | Code + nota + gambar |
| Run semua sekali gus | Run cell by cell |
| Susah nak experiment | Senang nak try & error |
| Output tak tersimpan | Output tersimpan! |

#### 🐍 IPython - The Brain Behind Jupyter

**IPython** = "Interactive Python" (dicipta 2001)
- Python biasa: Run sekali, habis
- IPython: Run line by line, tengok output terus!

**Jupyter** sebenarnya gabungan 3 bahasa:
- **Ju**lia + **Pyt**hon + **R** = **Jupyter**!

#### ☁️ Google Colab = Jupyter dalam Cloud

| Feature | Jupyter (Local) | Google Colab |
|---------|-----------------|--------------|
| Install | Kena install sendiri | Tak perlu! |
| GPU/TPU | Kena beli | FREE! 🎉 |
| Storage | PC korang | Google Drive |
| Share | Susah | Senang (macam Google Docs) |

> **Fun fact:** File `.ipynb` = **I**nteractive **Py**thon **N**ote**b**ook!

---

## Step 1: Setup - Install & Import Libraries

In [ ]:
# Suppress warnings FIRST (before any imports/installs)
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# ============================================================
# 📦 INSTALL LIBRARIES
# ============================================================
# Kalau guna Google Colab - libraries akan auto install
# Kalau guna local - run pip install fastai ddgs gradio
# ============================================================

!pip install -Uqq fastai ddgs gradio

print("✅ Libraries installed!")

## Step 2: Buat Function untuk Search Gambar

### 🔧 Apa Itu Function?

**Function** = "Mesin" yang buat kerja untuk kita!

Bayangkan **mesin jual air**:
1. Masukkan duit (INPUT)
2. Mesin proses
3. Keluar air (OUTPUT)

Function sama je:
1. Bagi data (INPUT/arguments)
2. Function proses
3. Dapat result (OUTPUT/return)

```python
# Struktur function:
def nama_function(input):
    # buat sesuatu
    return output
```

**Kenapa guna function?**
- ✅ Tak perlu tulis code sama berulang kali
- ✅ Code lebih kemas & organized
- ✅ Senang nak debug (cari error)

In [ ]:
# Contoh function mudah: Kira harga dengan diskaun

def kira_diskaun(harga, peratus_diskaun):
    """
    Function untuk kira harga selepas diskaun
    
    Input: harga asal, peratus diskaun
    Output: harga baru
    """
    diskaun = harga * (peratus_diskaun / 100)
    harga_baru = harga - diskaun
    return harga_baru

# Cuba function kita!
harga_asal = 100
diskaun = 20  # 20%

harga_final = kira_diskaun(harga_asal, diskaun)
print(f"Harga asal: RM{harga_asal}")
print(f"Diskaun: {diskaun}%")
print(f"Harga final: RM{harga_final}")

### 🔄 Apa Itu For Loop?

**For Loop** = Buat benda yang sama berulang kali!

Bayangkan korang jadi **cikgu** nak panggil nama semua pelajar:
- "Ali, hadir?"
- "Abu, hadir?"
- "Aminah, hadir?"
- ... (repeat untuk semua nama)

Dalam Python, tak perlu tulis satu-satu - guna **for loop**!

```python
# Struktur for loop:
for item in senarai:
    # buat sesuatu dengan item
```

In [ ]:
# Contoh for loop: Panggil nama pelajar

pelajar = ["Ali", "Abu", "Aminah", "Siti", "Ahmad"]

print("📋 Senarai kehadiran:")
print("=" * 30)

for nama in pelajar:
    print(f"✅ {nama}, hadir!")

print("=" * 30)
print(f"Jumlah pelajar: {len(pelajar)} orang")

### 🔀 Apa Itu If/Else?

**If/Else** = Buat keputusan berdasarkan syarat!

Bayangkan korang nak **pergi sekolah**:
- **Kalau** hujan → bawa payung
- **Kalau tidak** → tak payah bawa

Dalam Python, guna `if`, `elif` (else if), dan `else`:

```python
# Struktur if/else:
if syarat_1:
    # buat ni kalau syarat_1 TRUE
elif syarat_2:
    # buat ni kalau syarat_2 TRUE
else:
    # buat ni kalau semua syarat FALSE
```

**Operators untuk compare:**
| Operator | Maksud | Contoh |
|----------|--------|--------|
| `==` | Sama dengan | `x == 5` |
| `!=` | Tidak sama | `x != 5` |
| `>` | Lebih besar | `x > 5` |
| `<` | Lebih kecil | `x < 5` |
| `>=` | Lebih besar atau sama | `x >= 5` |
| `<=` | Lebih kecil atau sama | `x <= 5` |
| `in` | Ada dalam | `'a' in 'cat'` |

In [ ]:
# Contoh if/else: Check gred peperiksaan

def check_gred(markah):
    """Check gred berdasarkan markah"""
    if markah >= 90:
        gred = "A+"
        komen = "Cemerlang! 🌟"
    elif markah >= 80:
        gred = "A"
        komen = "Sangat baik! 👏"
    elif markah >= 70:
        gred = "B"
        komen = "Baik! 👍"
    elif markah >= 60:
        gred = "C"
        komen = "Sederhana"
    elif markah >= 50:
        gred = "D"
        komen = "Lulus"
    else:
        gred = "F"
        komen = "Kena study lagi! 📚"
    
    return gred, komen

# Test dengan pelbagai markah
print("📝 Keputusan Peperiksaan:")
print("=" * 40)

for markah in [95, 82, 75, 63, 55, 42]:
    gred, komen = check_gred(markah)
    print(f"Markah {markah}: Gred {gred} - {komen}")

### 🛡️ Apa Itu Try/Except?

**Try/Except** = "Plan B" kalau ada error!

Bayangkan korang nak **buka pintu**:
- **Try**: Cuba buka dengan kunci
- **Except**: Kalau kunci tak muat, ketuk pintu minta tolong!

Dalam coding, banyak benda boleh jadi error:
- Internet putus
- File tak wujud
- Server tak respond

**Try/Except** pastikan program tak crash - ada backup plan!

```python
# Struktur try/except:
try:
    # cuba buat sesuatu
except:
    # kalau error, buat ni instead
```

In [ ]:
# Contoh try/except: Bahagi nombor

def bahagi(a, b):
    """Bahagi a dengan b, dengan error handling"""
    try:
        result = a / b
        print(f"✅ {a} ÷ {b} = {result}")
        return result
    except ZeroDivisionError:
        print(f"❌ Error: Tak boleh bahagi dengan 0!")
        return None

# Test cases
print("Test 1:")
bahagi(10, 2)  # OK

print("\nTest 2:")
bahagi(10, 0)  # Error! Tapi program tak crash

print("\nTest 3:")
bahagi(100, 4)  # OK

print("\n✅ Program masih running walaupun ada error!")

### 📋 Apa Itu List?

**List** = Senarai yang boleh simpan banyak benda!

Bayangkan **beg sekolah**:
- Boleh masukkan buku, pensel, air...
- Semua dalam satu beg
- Boleh tambah atau buang bila-bila

Dalam Python, list guna kurungan segi empat `[ ]`

```python
# Struktur list:
senarai = [item1, item2, item3]
```

In [ ]:
# Contoh list: Senarai barang dalam beg

beg_sekolah = ["buku", "pensel", "air", "makanan"]

print("🎒 Isi beg sekolah:")
print(beg_sekolah)

# Akses item by index (mula dari 0)
print(f"\nItem pertama: {beg_sekolah[0]}")  # buku
print(f"Item terakhir: {beg_sekolah[-1]}")  # makanan

# Tambah item baru
beg_sekolah.append("phone")
print(f"\nSelepas tambah phone: {beg_sekolah}")

# Berapa banyak item?
print(f"Jumlah item: {len(beg_sekolah)}")

### 💪 Apa Itu L? (List dengan Superpower!)

**L** = List dari fastai yang ada **extra features**!

Bayangkan **beg sekolah biasa** vs **beg sekolah smart**:
- Beg biasa: Simpan barang je
- Beg smart: Boleh auto-sort, cari barang cepat, organize barang!

```python
# List biasa:
senarai = [1, 2, 3]

# L list (fastai):
senarai = L([1, 2, 3])
```

**Superpower L:**
- `.map()` - apply function ke semua item sekali gus
- `[0, 2]` - ambil multiple index serentak
- Built-in helpers yang senang guna

In [ ]:
# Contoh L list: Superpower list!
from fastcore.all import L

# Buat L list
pelajar = L(["ali", "abu", "aminah", "siti"])
print("📋 Senarai pelajar:")
print(pelajar)

# Superpower 1: .map() - tukar semua jadi UPPERCASE
print("\n💪 Superpower 1: .map()")
print(pelajar.map(str.upper))

# Superpower 2: Multiple indexing - ambil index 0 dan 2 sekali gus!
print("\n💪 Superpower 2: Multiple index")
print(pelajar[0, 2])  # ali dan aminah

# Superpower 3: Senang check length dan akses
print("\n💪 Superpower 3: Built-in helpers")
print(f"Total: {len(pelajar)} pelajar")
print(f"First: {pelajar[0]}, Last: {pelajar[-1]}")

### 🎯 Apa Itu Set?

**Set** = List yang **TAK BOLEH ADA DUPLICATE**!

Bayangkan **senarai kehadiran**:
- Kalau Ali dah sign in, tak boleh sign in lagi
- Setiap nama hanya boleh ada sekali je!

```python
# Struktur set:
senarai = set()           # Set kosong
senarai = {1, 2, 3}       # Set dengan items
```

**Guna set untuk:**
- ✅ Track benda yang dah ada (elak duplicate)
- ✅ Check membership cepat ("ada ke dalam set ni?")
- ✅ Remove duplicates dari list

In [ ]:
# Contoh set: Senarai kehadiran (tak boleh duplicate!)

kehadiran = set()  # Set kosong

# Tambah nama
kehadiran.add("Ali")
kehadiran.add("Abu")
kehadiran.add("Aminah")
print(f"Kehadiran: {kehadiran}")

# Cuba tambah Ali lagi - TAK JADI sebab dah ada!
kehadiran.add("Ali")
print(f"Selepas tambah Ali lagi: {kehadiran}")  # Still same!

# Check ada ke tak?
print(f"\nAli hadir? {'Ali' in kehadiran}")      # True
print(f"Siti hadir? {'Siti' in kehadiran}")    # False

# Guna set untuk remove duplicates dari list
list_dengan_duplicate = ["mangkuk", "buku", "mangkuk", "buku", "mangkuk"]
list_unik = list(set(list_dengan_duplicate))
print(f"\nSebelum: {list_dengan_duplicate}")
print(f"Selepas buang duplicate: {list_unik}")

### 🦆 Sekarang Kita Guna Semua Ni!

Okay, sekarang korang dah faham:
- ✅ **Function** - buat "mesin" untuk kerja tertentu
- ✅ **For Loop** - ulang benda sama banyak kali
- ✅ **If/Else** - buat keputusan berdasarkan syarat
- ✅ **Try/Except** - handle error supaya tak crash
- ✅ **List** - simpan banyak benda dalam satu senarai
- ✅ **L** - list dengan superpower (dari fastai)
- ✅ **Set** - list yang tak boleh ada duplicate

Jom combine semua untuk buat **image search function**!

---

**DuckDuckGo** ni search engine macam Google, tapi more privacy-friendly.

Kita buat function untuk search dan download gambar dari internet!

### 🔍 DuckDuckGo Search Syntax (Tips!)

Boleh guna syntax ni untuk search lebih specific:

| Syntax | Maksud | Contoh |
|--------|--------|--------|
| `"kata kata"` | Exact phrase | `"nasi lemak"` |
| `kata -exclude` | Exclude term | `mangkuk -plastik` |
| `kata +include` | Boost term | `buku +melayu` |
| `filetype:pdf` | File type | `recipe filetype:pdf` |
| `site:example.com` | Specific site | `cat site:wikipedia.org` |

📚 Full guide: https://duckduckgo.com/duckduckgo-help-pages/results/syntax

**Note:** Kita add retry logic sebab DuckDuckGo kadang-kadang rate limit kalau terlalu banyak request.

In [ ]:
# ============================================================
# 📥 IMPORT LIBRARIES
# ============================================================
# import = "ambil" code dari library untuk kita guna
#
# Analogi: Import macam "pinjam buku dari perpustakaan"
#          - Tak perlu tulis semua dari awal
#          - Guna je function yang dah ada
# ============================================================

# DDGS = DuckDuckGo Search - untuk cari gambar dari internet
from ddgs import DDGS

# fastcore = utility functions (helper tools)
# L = special list, Path = handle file paths
from fastcore.all import *

# fastai.vision.all = SEMUA tools untuk computer vision
# Includes: DataBlock, vision_learner, resnet18, etc.
from fastai.vision.all import *

# gradio = buat web interface/demo
import gradio as gr

# time = untuk delay/wait (elak rate limit)
import time

print("✅ Libraries imported!")

In [ ]:
# ============================================================
# 🎨 CUSTOM HTML PROGRESS TABLE untuk Google Colab
# ============================================================
# Colab ada issue dengan ipywidgets progress bar
# Kita buat custom HTML table yang render cantik!
# ============================================================
from IPython.display import display, HTML, clear_output
import time as time_module

class HTMLProgressCallback(Callback):
    """
    Custom callback yang display training progress sebagai HTML table.
    Cantik untuk presentation! 🎨
    """
    order = 60  # Run after other callbacks
    
    def __init__(self):
        self.results = []
        self.start_time = None
        self.epoch_start = None
        
    def before_fit(self):
        """Called before training starts"""
        self.results = []
        self.start_time = time_module.time()
        print("🏋️ Training started!")
        print("=" * 60)
        
    def before_epoch(self):
        """Called before each epoch"""
        self.epoch_start = time_module.time()
        
    def after_epoch(self):
        """Called after each epoch - update the HTML table"""
        # Calculate time for this epoch
        epoch_time = time_module.time() - self.epoch_start
        mins, secs = divmod(int(epoch_time), 60)
        time_str = f"{mins:02d}:{secs:02d}"
        
        # Get metrics
        epoch = self.epoch
        train_loss = self.recorder.losses[-1].item() if self.recorder.losses else 0
        
        # Get validation metrics
        valid_loss = self.recorder.values[-1][0] if self.recorder.values else 0
        error_rate_val = self.recorder.values[-1][1] if len(self.recorder.values[-1]) > 1 else 0
        
        # Store result
        self.results.append({
            'epoch': epoch,
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'error_rate': error_rate_val,
            'time': time_str
        })
        
        # Clear and redraw table
        clear_output(wait=True)
        self._display_table()
        
    def _display_table(self):
        """Render the HTML table"""
        html = """
        <style>
            .training-table {
                border-collapse: collapse;
                font-family: 'Segoe UI', Arial, sans-serif;
                font-size: 14px;
                margin: 10px 0;
                box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                border-radius: 8px;
                overflow: hidden;
            }
            .training-table th {
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 12px 20px;
                text-align: center;
                font-weight: 600;
            }
            .training-table td {
                padding: 10px 20px;
                text-align: center;
                border-bottom: 1px solid #eee;
            }
            .training-table tr:nth-child(even) { background-color: #f8f9fa; }
            .training-table tr:hover { background-color: #e8f4f8; }
            .training-table tr:last-child td {
                background-color: #e8f5e9;
                font-weight: 600;
            }
        </style>
        <div style="font-size:16px; font-weight:600; margin-bottom:10px;">🏋️ Training Progress</div>
        <table class="training-table">
            <tr><th>Epoch</th><th>Train Loss</th><th>Valid Loss</th><th>Error Rate</th><th>Time</th></tr>
        """
        for r in self.results:
            accuracy = (1 - r['error_rate']) * 100
            html += f"""<tr>
                <td style="color:#667eea;font-weight:600;">{r['epoch']}</td>
                <td style="color:#e74c3c;">{r['train_loss']:.4f}</td>
                <td style="color:#e74c3c;">{r['valid_loss']:.4f}</td>
                <td style="color:#f39c12;">{r['error_rate']:.4f} ({accuracy:.1f}%)</td>
                <td style="color:#27ae60;">{r['time']}</td>
            </tr>"""
        html += "</table>"
        
        if self.results:
            latest = self.results[-1]
            acc = (1 - latest['error_rate']) * 100
            total = time_module.time() - self.start_time
            m, s = divmod(int(total), 60)
            html += f"""<div style="margin-top:15px;padding:10px;background:#e8f5e9;border-radius:8px;">
                <b>📊 Status:</b> Epoch {latest['epoch']+1} done | <b>Accuracy: {acc:.1f}%</b> | Time: {m:02d}:{s:02d}
            </div>"""
        display(HTML(html))
    
    def after_fit(self):
        """Called after training completes"""
        if self.results:
            acc = (1 - self.results[-1]['error_rate']) * 100
            print(f"\n🎉 Training complete! Final accuracy: {acc:.1f}%")

# Disable broken default progress bar
import fastprogress
fastprogress.fastprogress.NO_BAR = True

print("✅ HTMLProgressCallback ready!")

In [ ]:
# ============================================================
# 🔍 SEARCH IMAGES FUNCTION
# ============================================================
# Function ni combine FOR LOOP + TRY/EXCEPT + IF/ELSE
# 
# FLOW DIAGRAM:
# ┌─────────────────────────────────────────────────────────┐
# │ FOR attempt in [0, 1, 2]:     ← Cuba 3 kali             │
# │   │                                                     │
# │   ├─► TRY: Search gambar      ← Cuba buat              │
# │   │     │                                               │
# │   │     ├─► Berjaya? → RETURN urls (keluar function)   │
# │   │     │                                               │
# │   │     └─► Gagal? → masuk EXCEPT                      │
# │   │                                                     │
# │   └─► EXCEPT: Ada error                                │
# │         │                                               │
# │         ├─► IF rate limit? → Wait, then continue loop  │
# │         │                                               │
# │         └─► ELSE (error lain)? → RETURN [] (give up)   │
# │                                                         │
# └─────────────────────────────────────────────────────────┘
# ============================================================

def search_images(term, max_images=30):
    """
    Search images dari DuckDuckGo with retry logic
    
    Args:
        term: Apa nak search (contoh: "nasi lemak")
        max_images: Berapa banyak gambar nak dapat
    
    Returns:
        List of image URLs
    """
    print(f"🔍 Searching for '{term}'...")
    
    # ========================================
    # FOR LOOP: Cuba 3 kali (attempt 0, 1, 2)
    # Kenapa? Sebab DuckDuckGo kadang-kadang 
    # rate limit, so kita retry!
    # ========================================
    for attempt in range(3):
        
        # ========================================
        # TRY: Cuba search gambar
        # Kalau berjaya, terus return (keluar)
        # Kalau gagal, masuk EXCEPT
        # ========================================
        try:
            # Guna DuckDuckGo API untuk search
            results = DDGS().images(query=term, max_results=max_images)
            
            # Extract URL dari setiap result
            # L() = special list dari fastai
            urls = L([r['image'] for r in results])
            
            print(f"   Found {len(urls)} images!")
            
            # BERJAYA! Return urls dan keluar dari function
            return urls
            
        # ========================================
        # EXCEPT: Kalau ada error, handle di sini
        # ========================================
        except Exception as e:
            
            # ========================================
            # IF/ELSE: Check jenis error
            # ========================================
            
            # IF: Error sebab rate limit (terlalu banyak request)
            if 'Ratelimit' in str(e) or '403' in str(e) or 'forbidden' in str(e).lower():
                # Kira wait time: attempt 0 = 15s, 1 = 30s, 2 = 45s
                wait_time = (attempt + 1) * 15
                print(f"   ⏳ Rate limited, waiting {wait_time}s...")
                time.sleep(wait_time)
                # Loop akan continue ke attempt seterusnya
                
            # ELSE: Error lain (bukan rate limit)
            else:
                print(f"   ⚠️ Error: {e}")
                # Give up terus, return empty list
                return L([])
    
    # ========================================
    # Kalau sampai sini = dah cuba 3 kali tapi 
    # masih rate limited. Give up!
    # ========================================
    print("   ❌ Failed after 3 attempts")
    return L([])

## Step 3: Download Dataset - Mangkuk vs Buku! 🥣

Sekarang kita download gambar untuk **2 kategori**:
- 🥣 **Mangkuk** (bowl)
- 📚 **Buku** (book)

Macam Jeremy Howard buat "bird or not bird" - kita buat "mangkuk ke bukan?"

### ⚖️ Kenapa Dataset Kena BALANCED?

| Imbalanced Dataset | Balanced Dataset |
|-------------------|------------------|
| Mangkuk: 19 images | Mangkuk: 30 images |
| Buku: 29 images | Buku: 30 images |
| AI mungkin **bias** ke buku! | AI belajar **sama rata**! |

---

### Step 3a: Setup Variables 📝

Mula-mula kita setup variables yang kita perlukan:

In [ ]:
# ============================================
# 📝 SETUP VARIABLES
# ============================================

# 1. Apa yang kita nak classify?
searches = ['mangkuk', 'buku']
print(f"Categories: {searches}")

# 2. Berapa gambar kita nak setiap category?
TARGET_IMAGES = 100
print(f"Target: {TARGET_IMAGES} images each")

# 3. Folder untuk simpan gambar
path = Path('mangkuk_or_not')

# Delete folder lama (kalau ada) - start fresh!
if path.exists():
    import shutil
    shutil.rmtree(path)
    print("🗑️ Deleted old folder")

# Buat folder baru
path.mkdir(exist_ok=True)
print(f"📁 Created folder: {path}")

# 4. Set untuk track URLs (elak duplicate)
all_downloaded_urls = set()
print(f"🎯 Ready to download!")

### Step 3b: Download Images 📥

Sekarang kita download gambar untuk setiap category.

**Apa yang code ni buat:**
1. Loop through setiap category (mangkuk, buku)
2. Search 100 URLs dari DuckDuckGo
3. Filter duplicate URLs
4. Download & resize gambar
5. Wait 10s sebelum next category (elak rate limit)

In [ ]:
# ============================================
# 📥 DOWNLOAD IMAGES
# ============================================

for i, category in enumerate(searches):
    print(f"📁 Category: {category.upper()}")
    print("=" * 40)
    
    # Buat subfolder untuk category ni
    dest = path / category
    dest.mkdir(exist_ok=True, parents=True)
    
    # Search 300 URLs (sebab ~30-50% akan fail)
    urls = search_images(f"{category} photo", max_images=300)
    
    if len(urls) > 0:
        # Filter URLs yang dah download (elak duplicate)
        new_urls = L([u for u in urls if u not in all_downloaded_urls])
        print(f"   New URLs (not duplicate): {len(new_urls)}")
        
        if len(new_urls) > 0:
            # Download gambar
            download_images(dest, urls=new_urls)
            
            # Resize untuk jimat space
            resize_images(dest, max_size=400, dest=dest)
            
            # Track URLs yang dah download
            all_downloaded_urls.update(new_urls)
    
    # Berapa banyak berjaya download?
    count = len(list(dest.glob('*')))
    print(f"   ✅ Downloaded: {count} images\n")
    
    # Wait sebelum next category (elak rate limit)
    if i < len(searches) - 1:
        print("⏳ Waiting 10s before next category...")
        time.sleep(10)

print("📥 Download phase complete!")

### Step 3c: Clean Data 🧹

Sebelum balance, kita kena **buang gambar rosak** dulu!

**Kenapa clean dulu?**
- Kalau balance dulu → clean → imbalanced balik! ❌
- Kalau clean dulu → balance → stays balanced! ✅

Fastai ada function `verify_images()` untuk check gambar yang corrupt/invalid.

In [ ]:
# ============================================
# 🧹 CLEAN DATA - Buang gambar rosak
# ============================================

print("🧹 CLEANING DATA...")
print("=" * 40)

# Get semua image files
image_files = get_image_files(path)
print(f"Total images before cleaning: {len(image_files)}")

# Check untuk corrupted/invalid images
failed = verify_images(image_files)

# Delete yang rosak
if len(failed) > 0:
    failed.map(Path.unlink)
    print(f"🗑️ Removed {len(failed)} corrupted images")
else:
    print("✅ No corrupted images found!")

# Count after cleaning
for cat in searches:
    count = len(list((path/cat).glob('*')))
    print(f"   {cat}: {count} images")

print("=" * 40)

### Step 3d: Balance Dataset ⚖️

Sekarang baru kita balance - buang yang lebih supaya semua category sama!

**Contoh:**
- Mangkuk: 42 images (selepas clean)
- Buku: 47 images (selepas clean)
- **Minimum = 42**
- Buku kena buang 5 images (47 → 42)

In [ ]:
# ============================================
# ⚖️ BALANCE DATASET
# ============================================
import random

print("⚖️ BALANCING DATASET...")
print("=" * 40)

# Kira berapa setiap category ada
counts = {}
for cat in searches:
    counts[cat] = len(list((path/cat).glob('*')))
print(f"Before: {counts}")

# Cari minimum
min_count = min(counts.values())
print(f"Target: {min_count} images each\n")

# Buang yang lebih
for category in searches:
    folder = path / category
    images = list(folder.glob('*'))
    current = len(images)
    
    if current > min_count:
        # Pilih random images untuk delete
        to_delete = random.sample(images, current - min_count)
        for img in to_delete:
            img.unlink()
        print(f"🗑️ {category}: Deleted {len(to_delete)} ({current} → {min_count})")
    else:
        print(f"✅ {category}: OK ({current} images)")

# Final report
print("\n" + "=" * 40)
final_counts = {cat: len(list((path/cat).glob('*'))) for cat in searches}
print(f"🎉 DONE! {final_counts}")
print(f"📊 Total: {sum(final_counts.values())} images")
print("✅ Dataset is BALANCED!" if len(set(final_counts.values())) == 1 else "⚠️ Still imbalanced")

## Step 4: Tengok Sample Gambar

Jom tengok gambar yang kita download!

In [ ]:
# Tengok beberapa gambar dari dataset
from PIL import Image as PILImg
import matplotlib.pyplot as plt

image_files = get_image_files(path)

if len(image_files) > 0:
    # Show up to 6 sample images
    n_show = min(6, len(image_files))
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    
    for idx in range(n_show):
        img = PILImg.open(image_files[idx])
        axes[idx].imshow(img)
        axes[idx].set_title(image_files[idx].parent.name if image_files[idx].parent.name != 'images' else image_files[idx].name[:20])
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_show, 6):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No images to display yet")

## Step 5: Sediakan Data untuk Training (DataLoaders)

### 🧠 Concept: DataBlock - "Recipe" untuk Data

> *"The DataBlock is the key thing you need to know about if you're going to know how to use different kinds of datasets."* - Jeremy Howard

**DataBlock** bagi fastai **semua info** yang diperlukan untuk prepare data. Jeremy Howard dan team dah analyze **ratusan projects** dan realize ada **5 benda je** yang perlu ditentukan:

| Parameter | Soalan yang Dijawab | Contoh Kita |
|-----------|---------------------|-------------|
| `blocks` | **Apa jenis input & output?** | Input = gambar, Output = category |
| `get_items` | **Macam mana nak dapat list items?** | `get_image_files` - scan folder |
| `splitter` | **Macam mana nak split train/validation?** | 80% train, 20% validation |
| `get_y` | **Macam mana nak dapat label?** | `parent_label` - nama folder |
| `item_tfms` | **Apa transformations per item?** | Resize semua jadi 224×224 |

### ❓ Kenapa Validation Set PENTING?

> *"It's so critical that fast.ai won't let you train a model without one."* - Jeremy Howard

**Validation Set** = Data yang AI **TAK PERNAH nampak** masa training!
- Guna untuk test **sebenar-benarnya** model tu pandai ke tak
- Kalau train dan test guna data SAMA = model just "menghafal", bukan "faham"!

**Analogi:** Macam **peperiksaan SPM**:
- Training data = Buku teks & latihan (boleh study berulang kali)
- Validation data = Soalan SPM sebenar (tak pernah nampak sebelum ni!)

Kalau student score A dalam homework tapi F dalam SPM = **overfitting** (menghafal, bukan faham)!

In [ ]:
# Buat DataLoaders
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    # 224px = standard size untuk pretrained models (ImageNet)
    item_tfms=Resize(224, method='squish'),
    # Data augmentation dengan parameter yang lebih specific
    batch_tfms=aug_transforms(
        do_flip=True,       # Flip horizontal
        flip_vert=False,    # Tak flip vertical (mangkuk terbalik = pelik!)
        max_rotate=10,      # Rotate max 10 degrees
        max_zoom=1.1,       # Zoom max 10%
        max_lighting=0.2    # Adjust brightness/contrast
    )
).dataloaders(path, bs=32, num_workers=0)  # num_workers=0 for Colab compatibility

print("✅ DataLoaders created!")
print(f"   Training batches: {len(dls.train)}")
print(f"   Validation batches: {len(dls.valid)}")
print(f"   Categories: {dls.vocab}")

In [ ]:
# Jom tengok data kita!
dls.show_batch(max_n=9)

### 🤔 Apa Maksud `max_n=9`?

`max_n=9` bermaksud **maximum number = 9** - tunjuk maksimum 9 gambar.

**Tapi kenapa kita nampak 9 gambar je, bukan lebih?**

Sebab **batch size = 32**! Tengok balik code kita:

```python
.dataloaders(path, bs=32, num_workers=0)
#                  ^^^^
#                  batch size = 32
```

`show_batch()` tunjuk **satu batch** je. Kalau kita minta `max_n=9`, dia tunjuk 9 gambar dari batch tu.

---

### 📊 Apa Maksud "Training batches: X"?

Output tadi kata `Training batches: X`. Ini bermaksud:

```
X batches × 32 images = total training images
```

**Contoh dengan ~200 total images:**
- Total images = ~200 (100 mangkuk + 100 buku)
- Training (80%) = ~160 images → **5 batches**
- Validation (20%) = ~40 images → **1-2 batches**

> 💡 `valid_pct=0.2` dalam `RandomSplitter` bermaksud 20% data jadi validation set.

### 🤷 Kenapa Kena Ada Batch?

Ada 3 sebab utama:

#### 1. 💾 Memory Limit (GPU/RAM)

```
❌ Load semua 96 gambar sekali gus = GPU MELETUP! (Out of memory)
✅ Load 8 gambar je sekali = GPU boleh handle
```

Macam **makan buffet** - tak boleh letak SEMUA makanan dalam pinggan sekali, ambil sikit-sikit!

#### 2. 🎯 Belajar Lebih Baik

| Kaedah | Pros | Cons |
|--------|------|------|
| **1 gambar sekali** | Update cepat | Terlalu "noisy", tak stable |
| **Semua gambar sekali** | Stable | Lambat, stuck di local minimum |
| **Batch (8-64)** | Best of both! ✅ | Sweet spot |

#### 3. 🔀 Randomness Membantu

Setiap epoch, batch di-**shuffle** (kocok). Ini bantu AI tak "menghafal" order gambar!

### 🔄 Macam Mana Training Berjalan?

AI train **batch by batch**, kemudian **shuffle** dan ulang!

```
EPOCH 1:
┌─────────────────────────────────────────────────────┐
│ Batch 1 → Train → Update weights (AI belajar)       │
│ Batch 2 → Train → Update weights                    │
│ Batch 3 → Train → Update weights                    │
│ ...                                                 │
│ Batch 12 → Train → Update weights                   │
└─────────────────────────────────────────────────────┘
                    ↓ SHUFFLE semua gambar! 🔀
        
EPOCH 2:
┌─────────────────────────────────────────────────────┐
│ Batch 1 → Train → Update (gambar dalam batch dah    │
│           lain sebab shuffle!)                      │
│ ...                                                 │
└─────────────────────────────────────────────────────┘
                    ↓ SHUFFLE lagi! 🔀

EPOCH 3: ... repeat ...
```

**Analogi:** Macam **latih tubi matematik**:
- Buat 8 soalan → cikgu betulkan → faham sikit
- Buat 8 soalan lagi → makin improve
- Habis semua soalan (1 epoch) → cikgu kocok soalan
- Buat semua lagi dengan order berbeza → makin pandai!

Sebab tu `error_rate` makin **turun** setiap epoch! 📉

---

## Step 6: Buat Model (AI Brain! 🧠)

### 🔄 The Machine Learning Training Loop

Sebelum kita buat model, faham dulu **macam mana AI belajar**:

```
┌─────────────────────────────────────────────────────────────┐
│                    MACHINE LEARNING LOOP                     │
│                                                             │
│   ┌─────────┐     ┌─────────┐     ┌─────────┐              │
│   │ INPUTS  │ ──► │  MODEL  │ ──► │ RESULTS │              │
│   │(gambar) │     │(weights)│     │(predict)│              │
│   └─────────┘     └─────────┘     └─────────┘              │
│                        ▲               │                    │
│                        │               ▼                    │
│                   ┌─────────┐     ┌─────────┐              │
│                   │ UPDATE  │ ◄── │  LOSS   │              │
│                   │ WEIGHTS │     │(how bad)│              │
│                   └─────────┘     └─────────┘              │
│                                                             │
│   Repeat sampai LOSS jadi rendah! 🔄                        │
└─────────────────────────────────────────────────────────────┘
```

> *"The model is a mathematical function... we start with weights as being random. So initially, this thing doesn't do anything useful at all! ...we need a way of updating the weights that is a bit better than the previous set."* - Jeremy Howard

**Langkah-langkah:**
1. **Input** gambar ke dalam **Model** (yang ada **Weights**)
2. Model hasilkan **Results** (prediction)
3. Compare dengan jawapan sebenar → dapat **Loss** (how wrong?)
4. **Update Weights** supaya next time kurang salah
5. **Repeat** sampai Loss rendah!

---

### 🧠 Concept: Transfer Learning & Pre-trained Models

> *"The reason we can do this so fast is because somebody else has already trained this model to recognize over 1 million images of over 1000 different types."* - Jeremy Howard

**ResNet18** bukan model kosong - ia dah **pre-trained**!

| Tanpa Pre-training | Dengan Pre-training |
|-------------------|---------------------|
| Start dari RANDOM weights | Start dari weights yang dah PANDAI |
| Kena train dengan JUTAAN gambar | Kena train dengan ~200 gambar je |
| Ambil masa DAYS/WEEKS | Ambil masa MINIT je! |
| Perlukan GPU mahal | Laptop biasa pun boleh! |

**Analogi:** 
- **Tanpa pre-training** = Ajar budak umur 3 tahun jadi doktor (kena start dari ABC!)
- **Dengan pre-training** = Ajar medical student jadi doktor (dah tau basics!)

### 🎯 Fine-Tuning = "Adjust sikit je"

`fine_tune()` dalam fastai:
1. **Freeze** semua layers (tak ubah weights yang dah pandai)
2. Train layer **terakhir** je dulu (untuk task baru kita)
3. **Unfreeze** dan train semua dengan learning rate **sangat kecil**

> *"It takes those pre-trained weights we downloaded for you, and adjusts them in a really carefully controlled way to just teach the model the differences between your dataset and what it was originally trained for."* - Jeremy Howard

---

### 🎨 Model Options:
| Model | Speed | Accuracy | Best For |
|-------|-------|----------|----------|
| `resnet18` | ⚡ Fast | Good | Beginners, quick demos |
| `resnet34` | Medium | Better | More accuracy needed |
| `resnet50` | Slower | Even better | Serious projects |
| `convnext_tiny` | Slow | Excellent | Competition/production |

### 📚 Resources untuk pilih model:
- **Jeremy Howard's Analysis:** https://www.kaggle.com/code/jhoward/which-image-models-are-best
- **timm Model Zoo:** https://huggingface.co/timm

> 💡 **Tip:** Untuk beginners, `resnet18` dah cukup bagus. Bila dah confident, cuba `convnext_tiny`!

In [ ]:
# Buat learner (model + training settings)
# Guna resnet18 - fast & good untuk beginners
learn = vision_learner(dls, resnet18, metrics=error_rate)

# ADVANCED: Nak accuracy lebih tinggi? Cuba models ni:
# learn = vision_learner(dls, resnet34, metrics=error_rate)
# learn = vision_learner(dls, 'convnext_tiny.fb_in22k', metrics=error_rate)

print("🧠 Model created! Ready to train!")

## Step 7: TRAIN! 🏋️

Ni part paling best - tengok AI belajar secara real-time!

Kita train untuk **15 epochs** (15 rounds through all images)

In [ ]:
# Train untuk 15 epochs dengan HTML progress table! 🎨
# Remove default ProgressCallback (broken di Colab) dan guna HTMLProgressCallback
from fastai.callback.progress import ProgressCallback
learn.remove_cb(ProgressCallback)
learn.fine_tune(15, cbs=[HTMLProgressCallback()])

### 🎉 Check Results!

Tengok **error_rate** - kalau rendah, meaning AI kita accurate!

Contoh: error_rate = 0.15 means **85% accurate**!

## Step 8: Tengok Predictions

Jom tengok macam mana AI kita perform!

In [ ]:
# Show predictions vs actual
learn.show_results(max_n=6)

In [ ]:
# Confusion Matrix - nak tengok mana yang AI paling confuse
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# Show top losses - images yang AI paling confuse
# Ni best untuk faham kenapa model buat silap!
interp.plot_top_losses(6, figsize=(12,8))

### 🧹 Optional: Clean Data yang Confusing

Guna `ImageClassifierCleaner` untuk review dan delete gambar yang tak betul!

In [ ]:
# OPTIONAL: Interactive widget untuk clean dataset
# Uncomment lines below untuk guna:

# from fastai.vision.widgets import *
# cleaner = ImageClassifierCleaner(learn)
# cleaner

In [ ]:
# After using cleaner widget, run this to delete selected images:

# for idx in cleaner.delete():
#     cleaner.fns[idx].unlink()
#     print(f"Deleted: {cleaner.fns[idx]}")

## Step 9: Export Model untuk Deployment

Save model supaya boleh deploy ke web!

In [ ]:
# Export model untuk deployment
# pickle_protocol=4 untuk compatibility across Python versions
learn.export('mangkuk_model.pkl', pickle_protocol=4)

print("✅ Model exported as 'mangkuk_model.pkl'")
print("📥 Download file ni untuk deploy ke Hugging Face!")

## Step 10: Buat Web Demo dengan Gradio! 🌐

Gradio = buat web interface untuk AI dalam beberapa line code je!

Run cell ni untuk dapat **public link** yang boleh share dengan kawan!

In [ ]:
# Load exported model
learn_inf = load_learner('mangkuk_model.pkl')
categories = learn_inf.dls.vocab

def classify_image(img):
    """Classify mangkuk or buku"""
    pred, pred_idx, probs = learn_inf.predict(img)
    return {categories[i]: float(probs[i]) for i in range(len(categories))}

# Create Gradio interface
demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="🥣 Mangkuk ke Bukan?",
    description="Upload gambar - AI akan kenal sama ada Mangkuk atau Buku!",
    examples=[[str(f)] for f in get_image_files(path)[:3]]  # Show 3 example images
)

# Launch with public link!
demo.launch(share=True)

## Step 11: Deploy ke Hugging Face Spaces (FREE!) 🚀

Nak model korang online **24/7**? Deploy ke Hugging Face Spaces!

### Cara Deploy:

1. **Buat akaun** di https://huggingface.co/join (FREE!)

2. **Create New Space:**
   - Click profile → "New Space"
   - Name: `mangkuk-classifier`
   - SDK: **Gradio**
   - Visibility: **Public**

3. **Upload 3 files:**

**File 1: `app.py`**
```python
from fastai.vision.all import *
import gradio as gr

learn = load_learner('mangkuk_model.pkl')
categories = learn.dls.vocab

def classify_image(img):
    pred, pred_idx, probs = learn.predict(img)
    return {categories[i]: float(probs[i]) for i in range(len(categories))}

demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="🥣 Mangkuk ke Bukan?",
    description="Mangkuk atau Buku?"
)
demo.launch()
```

**File 2: `requirements.txt`**
```
fastai
gradio
```

**File 3: `mangkuk_model.pkl`** (download dari Colab)

4. **Wait 2-3 minit** untuk build, then DONE! 🎉

Your URL: `https://huggingface.co/spaces/YOUR_USERNAME/mangkuk-classifier`

In [ ]:
# Test dengan gambar dari dataset
test_files = get_image_files(path)
if len(test_files) > 0:
    # Pick random image
    import random
    test_img = PILImage.create(random.choice(test_files))
    
    # Predict!
    pred, pred_idx, probs = learn.predict(test_img)
    
    print(f"🔮 AI kata: {str(pred).upper()}")
    print(f"📊 Confidence: {probs[pred_idx]:.1%}")
    test_img.to_thumb(256, 256)
else:
    print("⚠️ No test images available")

In [ ]:
# === TEMPLATE: Buat Classifier Sendiri! ===

# 1. Define categories korang (2 categories macam Jeremy Howard's "bird or not bird")
my_searches = ['bird photo', 'cat photo']  # <-- TUKAR NI!

# 2. Create folder
my_path = Path('my_classifier')

# 3. Download images (dengan delay untuk elak rate limit)
for search_term in my_searches:
    folder_name = search_term.split()[0]  # First word as folder name
    dest = my_path / folder_name
    dest.mkdir(exist_ok=True, parents=True)
    
    # Search 300 images (sebab ~30-50% akan fail)
    urls = search_images(search_term, max_images=300)
    if len(urls) > 0:
        download_images(dest, urls=urls)
    
    time.sleep(5)  # Wait 5s between searches!
    print(f"✅ {folder_name} done!")

# 4. Clean data
failed = verify_images(get_image_files(my_path))
failed.map(Path.unlink)
print(f"\n📁 Total images: {len(get_image_files(my_path))}")

In [ ]:
# 5. Create DataLoaders
my_dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(192)
).dataloaders(my_path, bs=32, num_workers=0)  # num_workers=0 for Colab

# 6. Show sample
my_dls.show_batch(max_n=9)

In [ ]:
# 7. Train model!
my_learn = vision_learner(my_dls, resnet18, metrics=error_rate)
my_learn.remove_cb(ProgressCallback)
my_learn.fine_tune(15, cbs=[HTMLProgressCallback()])

In [ ]:
# 8. Check results
my_learn.show_results(max_n=6)

---

## 🎯 Apa Deep Learning Boleh Buat?

> *"Nearly always they say: 'Well we just broke the state-of-the-art results in our field.'"* - Jeremy Howard

Deep learning sekarang **state-of-the-art** untuk:

| Domain | Contoh Applications |
|--------|---------------------|
| **Computer Vision** | Image classification, object detection, face recognition |
| **NLP** | Translation, chatbots, sentiment analysis |
| **Medicine** | Cancer detection, drug discovery, medical imaging |
| **Recommendation** | Netflix, Spotify, TikTok FYP |
| **Games** | AlphaGo, game AI, procedural generation |
| **Audio** | Speech recognition, music generation |

### 💡 Idea Projects dari Fast.ai Alumni!

Jeremy Howard share banyak projects dari students dalam course.fast.ai:

| Project | Description | Result |
|---------|-------------|--------|
| **Trinidad & Tobago People Classifier** | Classify different types of people | Fun cultural project! |
| **Satellite City Classifier** | Which city is this satellite image from? | 85% accuracy with 110 classes! |
| **Panama City Bus Classifier** | Recognize different buses | Useful for locals! |
| **Sound Classifier** | Convert sound → image → classify | State-of-the-art result! 🏆 |
| **Mouse Movement Fraud Detector** | Convert mouse movements to images | Used at Splunk (became patent!) |
| **Cancer Detection** | Tumor vs normal tissue | Medical research impact! |

> *"Many of these have ended up turning into new startups, scientific papers, job offers."* - Jeremy Howard

---

## 💡 Discussion Questions

1. **Kenapa AI kadang-kadang confuse?**
   - Gambar dari internet kadang-kadang tak berkualiti
   - Background/lighting yang sama antara categories
   - Need more training data
   - Images yang "menipu" (contoh: buku berbentuk mangkuk)

2. **Macam mana nak improve accuracy?**
   - Download lebih banyak gambar (100+ setiap category)
   - Manually check dan buang gambar yang tak betul
   - Train lebih lama (5+ epochs)
   - Guna model lebih besar (resnet34, resnet50)
   - Data augmentation yang betul

3. **Creative ideas untuk image classifiers:**
   - **Convert non-image data to images!**
     - Sound → Spectrogram → Image classifier (state-of-the-art!)
     - Time series → Line chart → Image classifier
     - Mouse movements → Pattern image → Fraud detection

---

## 🏠 Homework Ideas

> *"Have a go at starting something. Create something you think would be fun or interesting and share it in the forum."* - Jeremy Howard

### Level 1: Beginner
- [ ] Classify Malaysian food (nasi lemak vs roti canai)
- [ ] Classify animals (kucing vs ayam)
- [ ] Classify objects in your room

### Level 2: Intermediate  
- [ ] Classify different types of cars (Myvi vs Vios)
- [ ] Classify healthy vs unhealthy food
- [ ] Classify batik patterns

### Level 3: Advanced
- [ ] Classify satellite images of different states
- [ ] Convert sounds to spectrograms and classify
- [ ] Build recommendation system for local products

---

## 📚 Resources

- **course.fast.ai** - Full free deep learning course by Jeremy Howard
- **Kaggle.com** - Download datasets dan compete
- **Hugging Face** - Deploy models for free
- **timm** - Huge collection of computer vision models

### 📖 Read the Book!
> *"The educational research literature shows that people learn things best when they hear the same thing in multiple different ways."* - Jeremy Howard

Book: **"Deep Learning for Coders with fastai & PyTorch"**
- Free online di GitHub: github.com/fastai/fastbook
- Sama content, different presentation = better learning!

---

## 🎯 Key Takeaways

1. **Deep learning dah jadi accessible** - Laptop biasa pun boleh!
2. **Transfer learning = superpower** - Guna pre-trained models
3. **Data quality > Data quantity** - Better clean data than more messy data
4. **DataBlock = Recipe** - 5 things je perlu specify
5. **Validation set is critical** - To know if model is actually learning
6. **Neural networks learn features automatically** - No need to code edge detectors!
7. **Fine-tune, don't train from scratch** - Much faster, better results!

---

*"The future belongs to those who learn AI. And that's YOU!"* 🚀🇲🇾

*Sekarang korang dah boleh train AI dengan data sendiri dari internet! That's powerful!*